In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import imutils
import random

In [ ]:
# def boundingRect(cnt):
#     x_list = [p[0][0] for p in cnt]
#     y_list = [p[0][1] for p in cnt]
#     x1, x2 = min(x_list), max(x_list)
#     y1, y2 = min(y_list), max(y_list)
#     return (x1, y1), (x2, y2)

# data_folder = "../download_from_drive/data/O7/"
# img_fnms = sorted([fn for fn in os.listdir(data_folder) if fn.ends('.jpg')])

# for fn in random.sample(img_fnms, 20):
#     path = os.path.join(data_folder, fn)
#     img = cv2.imread(path)
#     orig = img.copy()
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     gray = cv2.GaussianBlur(gray, (7, 7), 0)
#     edged = cv2.Canny(gray, 75, 200)
#     cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
#     cnts = sorted(cnts, key=lambda x: cv2.arcLength(x, True), reverse=True)[:5]
    
#     has_cnt = False
    
#     for c in cnts:
        
#         # approximate the contour
#         peri = cv2.arcLength(c, True)
#         approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
#         # if our approximated contour has four points, then we
#         # can assume that we have found our screen
#         if len(approx) == 4:
#             (x1, y1), (x2, y2) = boundingRect(cnt=approx)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 14)
#             plt.figure(figsize=(12,12)); plt.imshow(img); plt.axis('off'); plt.show()
#             has_cnt = True
    
#             break
        
#     if not has_cnt:
#         print("NO CONTOUR FOR", path)        

In [ ]:
class Line:
    
    def __init__(self, pt1, pt2):
        self.pt1 = pt1
        self.pt2 = pt2
        self.__split_points()
        self.__compute_hvType()
        self.__compute_id_coord()
        
    def __split_points(self):
        self.x1, self.y1 = self.pt1
        self.x2, self.y2 = self.pt2
    
    def __compute_hvType(self):
        x_dist, y_dist = abs(self.x2 - self.x1), abs(self.y2 - self.y1)
        self.hv_type = 'v' if x_dist < y_dist else 'h'
        
    def __compute_id_coord(self):
        self.id_coord = np.mean([self.x1, self.x2]) if self.hv_type == 'v' else np.mean([self.y1, self.y2])
        self.id_coord = int(self.id_coord)
        
    def to_list(self):
        return self.x1, self.y1, self.x2, self.y2
    
        
def parse_object(obj):
    (x1, y1), (x2, y2) = obj['points']['exterior']
    return x1, y1, x2, y2
    
    
def load_json(fpath):
    lines = []
    objs = json.load(open(fpath, "r"))['objects']
    for obj in objs:
        if obj['geometryType'] == "line":
            x1, y1, x2, y2 = parse_object(obj)
            lines.append(Line(pt1=(x1, y1), pt2=(x2, y2)))
    return lines

In [ ]:
ANN_DIR = "./annotations/Batch1/ds0/ann/"
IMG_DIR = "../download_from_drive/data/ProcessedO7/"

In [ ]:
images_data_dict = {k: [] for k in ["img_key", "w", "h", "n_lines", "n_lines_v", "n_lines_h"]}

ann_files = [f for f in os.listdir(ANN_DIR) if f.endswith(".json")]
for ann_fn in ann_files:
    lines = load_json(fpath=os.path.join(ANN_DIR, ann_fn))
    images_data_dict['img_key'].append(ann_fn.split('.jpg.json')[0])
    images_data_dict['n_lines'].append(len(lines))
    h, w = cv2.imread(os.path.join(IMG_DIR, ann_fn.split('.json')[0])).shape[:2]
    images_data_dict['w'].append(w)
    images_data_dict['h'].append(h)
    vlines = [l for l in lines if l.hv_type == 'v']
    hlines = [l for l in lines if l.hv_type == 'h']
    images_data_dict["n_lines_v"].append(len(vlines))
    images_data_dict["n_lines_h"].append(len(hlines))

In [ ]:
images_df = pd.DataFrame(images_data_dict)
display(images_df)

# for n_lines_v in images_df.n_lines_v.unique():
#     print("n_lines_v =", n_lines_v)
#     for _, row in images_df.query("n_lines_v == @n_lines_v").sample(10).iterrows():
#         img = cv2.imread(os.path.join(IMG_DIR, row.img_key + ".jpg"))
#         plt.figure(figsize=(12,12)), plt.imshow(img); plt.axis('off'); plt.show()
#     print("\n\n\n")

In [ ]:
# N_LINES_V = 5
# lines_data_dict = {k: [] for k in ["img_key", "hv", "id_coord"]}

# df = images_df.query("n_lines_v == @N_LINES_V")
# for _, row in df.iterrows():
#     lines = load_json(fpath=os.path.join(ANN_DIR, row.img_key + ".jpg.json"))
    
#     for line in lines:
#         lines_data_dict["img_key"].append(row.img_key)
#         lines_data_dict["hv"].append(line.hv_type)
#         id_coord = line.id_coord / (row.w if line.hv_type == 'v' else row.h)
#         lines_data_dict["id_coord"].append(round(id_coord, 2))

# lines_df = pd.DataFrame(lines_data_dict)
# lines_df

In [ ]:
# d = {i: [] for i in range(N_LINES_V)}

# for img_key in lines_df.img_key.unique():
#     df = lines_df.query("hv == \"v\" and img_key == @img_key").sort_values("id_coord")
#     i = 0
#     for _, row in df.iterrows():
#         d[i].append(row.id_coord)
#         i += 1

# vlines_df = pd.DataFrame(d)
# for i in range(N_LINES_V):
#     vlines_df[i].hist()
#     plt.show()

In [ ]:
VLINES4_DICT = {
    0: 0.11,
    1: 0.16,
    2: 0.36,
    3: 0.55
}

VLINES5_DICT = {
    0: 0.12,
    1: 0.17,
    2: 0.33,
    3: 0.47,
    4: 0.54
}

In [ ]:
def get_lines_df(images_df, n_vlines):
    lines_data_dict = {k: [] for k in ["img_key", "hv", "id_coord"]}
    df = images_df.query("n_lines_v == @n_vlines")
    for _, row in df.iterrows():
        lines = load_json(fpath=os.path.join(ANN_DIR, row.img_key + ".jpg.json"))
        for line in lines:
            lines_data_dict["img_key"].append(row.img_key)
            lines_data_dict["hv"].append(line.hv_type)
            id_coord = line.id_coord / (row.w if line.hv_type == 'v' else row.h)
            lines_data_dict["id_coord"].append(round(id_coord, 2))
    lines_df = pd.DataFrame(lines_data_dict)
    return lines_df

lines4_df = get_lines_df(images_df=images_df, n_vlines=4)
lines5_df = get_lines_df(images_df=images_df, n_vlines=5)

In [ ]:
display(lines4_df.head(15))
display(lines5_df.head(15))

In [ ]:
def plot_lines(images_df, lines_df, lines_dict, n_samples=None, figsize=(15, 15)):
    tmp_df = images_df.copy()
    if n_samples is not None:
        tmp_df = tmp_df.sample(n=n_samples).reset_index().drop("index", axis=1)
#     display(tmp_df)
  
    for _, row in tmp_df.iterrows():
        img = cv2.imread(os.path.join(IMG_DIR, row.img_key + ".jpg"))
        w, h = row.w, row.h
        coords = [int(coord * w) for _, coord in lines_dict.items()]
        for coord in coords:
            img = cv2.line(img, (coord, 0), (coord, h - 1), (0,0,255), 5)
        plt.figure(figsize=figsize)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

In [ ]:
plot_lines(images_df.query("n_lines_v == 5"), lines5_df, VLINES5_DICT, 50)